# Demo 5: Complete Workflow

End-to-end: Spectral scoring → Holographic refinement → Sublinear optimization on 5k candidates.

**Key Concepts:**
- Chain all modules.
- Find top-50 with full stats.
- Summarize techniques used.

In [ ]:
import numpy as np
from workbench import (
    ZetaFiducials, SpectralScorer,
    holographic_refinement, normalize_signal,
    SublinearOptimizer
)

print('=' * 70)
print('DEMO 5: COMPLETE WORKFLOW')
print('=' * 70)
print('Task: Find best 50 candidates from 5000 using all techniques')
print()

In [ ]:
# Setup
n = 5000
candidates = np.arange(n)

# Step 1: Spectral scoring
print('Step 1: Spectral scoring with zeta zeros...')
zeros = ZetaFiducials.get_standard(15)
scorer = SpectralScorer(frequencies=zeros, damping=0.05)
spectral_scores = scorer.compute_scores(candidates, shift=0.05, mode='real')
print('  ✓ Computed spectral scores')

In [ ]:
# Step 2: Define reference baseline
print('\nStep 2: Computing reference baseline...')
reference = 1.0 / (np.log(candidates + 2) + 1e-12)
reference = normalize_signal(reference, method='max')
print('  ✓ Reference baseline ready')

In [ ]:
# Step 3: Holographic refinement
print('\nStep 3: Holographic refinement...')
refined_scores = holographic_refinement(
    spectral_scores,
    reference,
    method='hilbert',
    blend_ratio=0.6
)
print('  ✓ Scores refined using phase retrieval')

In [ ]:
# Step 4: Sublinear optimization
print('\nStep 4: Sublinear optimization...')
optimizer = SublinearOptimizer(use_holographic=False)  # Already refined

def score_fn(cands):
    return refined_scores[cands]

top_50, stats = optimizer.optimize(
    candidates,
    score_fn,
    top_k=50
)
print('  ✓ Optimized to top 50')

In [ ]:
# Results
print('\n' + '-' * 70)
print('RESULTS')
print('-' * 70)
print(f'Original candidates: {stats.n_original:,}')
print(f'Final candidates: {stats.n_final}')
print(f'Reduction: {(1 - stats.reduction_ratio) * 100:.1f}%')
print(f'Complexity: {stats.complexity_estimate}')
print(f'Time: {stats.time_elapsed:.4f}s')
print()
print(f'Top 10 results: {top_50[:10]}')
print()
print('Techniques used:')
print('  ✓ Spectral scoring (zeta zeros)')
print('  ✓ Phase retrieval (Hilbert transform)')
print('  ✓ Holographic refinement (interference)')
print('  ✓ Sublinear optimization (O(√n))')

print('=' * 70)

**Insight:** Full pipeline: 5k → 50 (99% reduction), zeta + Hilbert unlock structured insights.